In [3]:
import torch

from einops import einsum,rearrange
b, h, s, d = 2, 3, 4, 5

q = torch.randn(b, h, s, d)

k = torch.randn(b, h, s, d)
o_matmul = torch.matmul(q, k.transpose(-2, -1))  #einsum('b h s d, b h s d -> b h s s', q, k) 但这实际上会计算每个位置s的d维的点积，导致结果形状为b h s s
# 使用einops.einsum
k_t=rearrange('b h s d -> b h d s',k)
o_einsum = einsum('b h s1 d, b h d s2 -> b h s1 s2', q, k_t)
x, y, z = torch.randn(3, 20, 20, 20)
# output = einsum(x, y, z, "a b c, c b d, a g k -> a b k")

# 检查是否相同

# print(torch.allclose(o_einsum, o_matmul))  # 应该输出True
print(o_matmul.shape)

RuntimeError: Tensor type unknown to einops <class 'str'>

In [4]:
import torch

i_a = torch.randn(16, 32, 4, 8)
i_b = torch.randn(16, 32, 8, 16)

out = torch.einsum('b h i j, b h j d -> b h i d', i_a, i_b)
print(out.shape)


torch.Size([16, 32, 4, 16])


In [1]:
# 手撕 MHA, 多头注意力，注意的点是投影矩阵
# 使用 einops 这样好得多
from torch import nn
import torch
import einops
#  einops
class MHA(torch.nn.Module):# 注意是    Module
    def __init__(self,hidden_size,head_num):
        super().__init__()
        assert hidden_size% head_num==0
        self.hidden_size=hidden_size
        self.head_num=head_num
        self.q_linear=torch.nn.Linear(hidden_size,hidden_size)
        self.k_linear=torch.nn.Linear(hidden_size,hidden_size)
        self.v_linear=torch.nn.Linear(hidden_size,hidden_size)
        self.o_linear=torch.nn.Linear(hidden_size,hidden_size)
    def forward(self,x,mask=None):# 注意不要把向量写错了
        head_num=self.head_num
        q=self.q_linear(x)
        k=self.k_linear(x)
        v=self.v_linear(x)
        q=einops.rearrange(q,pattern="b s (h d) -> b h s d",h=head_num)# d=head dim
        k=einops.rearrange(k,pattern='b s (h d) -> b h s d',h=head_num)
        v=einops.rearrange(v,pattern='b s (h d) -> b h s d',h=head_num)
        atten_score=torch.matmul(q,k.transpose(-1,-2))/torch.sqrt(torch.tensor(self.hidden_size))# 注意 需要转为tensor才能 sqrt
        if mask!=None:# b s s
            mask = mask.unsqueeze(1)  # 在第一个维度上增加头维度  b 1 s s
            atten_score = atten_score.masked_fill(mask == 0, float('-inf'))
        atten_probs=torch.softmax(input=atten_score,dim=-1)
        # o= einops.einsum('b h s1 s2, b h s2 d -> b h s1 d',atten_probs,v)
        o= torch.matmul(atten_probs,v)# 矩阵乘法还是matmul最舒服，注意不符合交换律哦
        # 而且注意是 
        o= einops.rearrange(o,pattern='b h s d -> b s (h d)',h=head_num)
        return self.o_linear(o)
     


In [ ]:
def test_mha():
    hidden_size = 64
    head_num = 8
    batch_size = 2
    seq_len = 10
    
    # 初始化模型
    model = MHA(hidden_size, head_num)
    
    # 测试正常输入
    x = torch.randn(batch_size, seq_len, hidden_size)
    output = model(x)
    assert output.shape == (batch_size, seq_len, hidden_size), "输出形状错误"
    
    # 测试掩码 # 注意掩码是 B S S 哦
    mask = torch.ones(batch_size, seq_len, seq_len)
    mask[:, :, 5:] = 0  # 屏蔽后5个位置
    masked_output = model(x, mask)
    
    # 验证输出是否不同（简单检查）
    assert not torch.allclose(output, masked_output), "掩码未生效"
    
    print("所有测试通过！")

test_mha()


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "d:\TOOL\Anaconda\Anaconda_root\envs\torchok\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "d:\TOOL\Anaconda\Anaconda_root\envs\torchok\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "d:\TOOL\Anaconda\Anaconda_root\envs\torchok\Lib\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_lo

所有测试通过！
